In [37]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd

In [39]:
data = pd.read_csv('wpbc.data', header=None)

In [278]:
def U(theta, X, y, Sigma_beta):
    a = 0
    for i in range(len(y)):
        a += (X[i] @ theta * y[i] - np.log(1+np.exp(X[i] @ theta)))
    return -a + 0.5 * theta.T @ np.linalg.inv(Sigma_beta) @ theta

In [616]:
def gradU(theta, X, y, Sigma_beta):
    a = 0
    for i in range(len(y)):
        a += (X[i] * y[i] - (X[i] * np.exp(X[i] @ theta)) / (1 + np.exp(X[i] @ theta)))
    return -a + np.linalg.inv(Sigma_beta) @ theta

In [623]:
bb = np.array([0,0,0,0,0]).astype(np.float64)

In [625]:
def hmc_vector(U, gradU, m, dt, nstep, x, X, y, Sigma_beta):
    mean = np.zeros_like(x)
    p = np.random.multivariate_normal(mean, m)
    oldX = x.copy()
    oldEnergy = 0.5 * p.dot(np.linalg.solve(m, p))  + U(x, X, y, Sigma_beta)
    for i in range(nstep):
        p -= gradU(x, X, y, Sigma_beta) * dt/2.
        x += np.linalg.solve(m, p) * dt
        p -= gradU(x, X, y, Sigma_beta) * dt/2.
    newEnergy = 0.5 * p.dot(np.linalg.solve(m, p))  + U(x, X, y, Sigma_beta)
    if np.random.random() > np.exp(oldEnergy - newEnergy): # Metropolis-Hastings
        x = oldX.copy()
    return x.copy()

In [636]:
nsample = 8000
m = np.eye(5)
dt = 0.1
nstep = 10

In [637]:
samples = []
bb = np.zeros_like(bb)
for i in range(nsample):
    bb = hmc_vector(U, gradU, m, dt, nstep, bb, XX, y, Sigma_beta)
    print(i)
    samples.append(bb)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [638]:
samples

[array([-0.14659022, -0.07427706,  2.29859274, -0.44191583, -1.0607591 ]),
 array([ 3.02008622, -0.45776819,  0.31178631, -0.50629131, -0.78098922]),
 array([ 0.06191124,  0.14304142,  0.35156714,  0.27674146, -0.28277369]),
 array([ 2.73288892, -0.7353897 ,  1.01981203,  0.35181223, -1.78852026]),
 array([ 0.42619007, -1.08276955,  0.31264981,  0.40211081,  0.6337782 ]),
 array([ 2.80457168, -1.04408895,  0.11388317,  0.63517138, -0.69500651]),
 array([ 0.16630128,  1.18564913, -0.08391094,  0.82222992, -1.03506858]),
 array([ 2.66646952, -0.08328788, -0.20956626,  1.14202121, -1.61779744]),
 array([ 0.16147251,  0.85220546,  0.77549966, -0.24992861, -0.52750073]),
 array([ 2.6407006 , -0.5233436 ,  0.95862457,  0.0534741 , -1.18856368]),
 array([ 0.20608615, -0.01078237,  1.05698299,  0.12133193, -0.30856948]),
 array([ 2.71727302, -0.65872936,  1.35100459, -1.37987173, -0.08096815]),
 array([ 0.57340093,  0.6934282 ,  0.44536005,  0.12141351, -0.54497824]),
 array([ 2.09249123,  0.6

In [640]:
samples = np.array(samples)
samples.mean(axis=0)

array([ 1.35263749, -0.2070946 ,  0.96616421, -0.2160136 , -0.41400247])

In [2]:
%load_ext rpy2.ipython

In [ ]:
%%R -i XX,y

library(R2jags)
dat = read.csv('wpbc.csv',header=F)


jags_data = list(Y = y,
                 X = XX,
                 n = length(y),
                 p = ncol(XX))

jags.model = "model {
  
  for (i in 1:n) {
    logit(mu[i]) = inprod(X[i,], theta)
    Y[i] ~ dbern(mu[i])
  }
  
  for (j in 1:(p)){
    theta[j] ~ dnorm(0,1)
  }
}
"

parameters = "theta"

jags_sim = jags(jags_data, inits=NULL, 
                parameters.to.save=parameters,
                model.file=textConnection(jags.model),
                n.iter=10000)

jags_sim

```
> jags_sim
Inference for Bugs model at "9", fit using jags,
 3 chains, each with 10000 iterations (first 5000 discarded), n.thin = 5
 n.sims = 3000 iterations saved
         mu.vect sd.vect    2.5%     25%     50%     75%   97.5%  Rhat
theta[1]   1.347   0.289   0.808   1.155   1.340   1.532   1.959 1.002
theta[2]  -0.241   0.829  -1.868  -0.804  -0.240   0.299   1.382 1.001
theta[3]   0.973   0.551  -0.089   0.588   0.962   1.324   2.091 1.002
theta[4]  -0.220   0.841  -1.831  -0.777  -0.237   0.345   1.436 1.003
theta[5]  -0.394   0.629  -1.662  -0.810  -0.399   0.036   0.831 1.002
deviance 186.465   2.588 183.346 184.731 185.909 187.563 192.821 1.001
         n.eff
theta[1]  1800
theta[2]  3000
theta[3]  1500
theta[4]   800
theta[5]  1900
deviance  3000
```